## Dynamic Behavior of a Systems

### Downloads

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 59.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
# PlantUML 
!pip install plantuml

### Imports

In [2]:
# Spacy Imports
import spacy 
from spacy.lang.en import English

# TextBlob Imports 
from textblob import TextBlob

# nltk Imports 
import nltk
from nltk.corpus import stopwords 
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

#plantUML Imports 
import plantuml
from plantuml import PlantUML

# Other Imports 
import string
from os.path import abspath 
import collections
from collections import defaultdict

### nltk Downloads

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

### Creating the function that Generate the PlantUML Code 

In [ ]:
text= ("  When the condition Primary_Side is true, it transits from Start state to Pilot_Flying state."
         "  When the condition Primary_Side is true, it transits from Start state to Pilot_Flying state."
         "  When the condition Primary_Side is false, it transits from Start state to Inhibited state."
         "  The variable Inhibit_count is initialized to "Zero" at the Inhibited state."
         "  The variable Inhibit_count is updated to "Inhibit_count++" at the Inhibited state."
         "  When the condition inhibit_count>=2, it transfers from Inhibited state to Listening state."
         "  When the event Transfer_Switch occurs, transition takes place from Listening state to the Pilot_Flying state."
         "  When the event Other_Side_Pilot_Flying occurs, the transits from the Pilot_Flying state to the Inhibited state.")

In [4]:
modified_text = ( " When the condition Primary_Side is true, it transits from the Start state to the Pilot_Flying state."
                  " When the condition Primary_Side is false, it transits from the Start state to the Inhibited state."
                  " The variable Inhibit_count is initialized to Zero at the Inhibited state."
                  " The variable Inhibit_count is updated to Inhibit_count++ at the Inhibited state."
                  " When the condition Inhibit_count>=2 is true, it transfers from the Inhibited state to the Listening state."
                  " When the Transfer_Switch event occurs, the Listening state transitions to the Pilot_Flying state."
                  " When the Other_Side_Pilot_Flying event occurs, the Pilot_Flying transitions to the Inhibited state.")

In [5]:
# Checking if it is tokenizing every sentence 
sent_tokenize(modified_text)

[' When the condition Primary_Side is true, it transits from the Start state to the Pilot_Flying state.',
 'When the condition Primary_Side is false, it transits from the Start state to the Inhibited state.',
 'The variable Inhibit_count is initialized to Zero at the Inhibited state.',
 'The variable Inhibit_count is updated to Inhibit_count++ at the Inhibited state.',
 'When the condition Inhibit_count>=2 is true, it transfers from the Inhibited state to the Listening state.',
 'When the Transfer_Switch event occurs, the Listening state transitions to the Pilot_Flying state.',
 'When the Other_Side_Pilot_Flying event occurs, the Pilot_Flying transitions to the Inhibited state.']

In [7]:
def tokenAndPOS_Tags(sent): 
  nlp = English()
  token_sentence = nlp(sent)
  # Allows the tokenizing of special words, this way words containing all 
  # symbols outside of the ones outlined are not seperated
  # Eg: Inhibited_count>=2
  suffixes = nlp.Defaults.suffixes + [r"\-|\|\$",]
  suffix_regex = spacy.util.compile_suffix_regex(suffixes)
  nlp.tokenizer.suffix_search = suffix_regex.search

  doc = nlp(sent)
  token_sentence = []
  # Extract the token and store them in the token_sentence list
  for token in doc:
    token_sentence.append(token.text)

  # POS Tag the tokens 
  pos_tag_token = nltk.pos_tag(token_sentence)
  
  return token_sentence, pos_tag_token

In [8]:
# Noun Extraction functions 
def nounExtraction(tag_tokens):
  # Noun Extraction (only extracting Proper Nouns)
  # Search each tag, if the tag is NNP, append that word
  # to the list of nouns (nltk_noun)
  nltk_nouns = []
  for index,tuple in enumerate(tag_tokens):
    if tuple[1] == 'NNP':
      nltk_nouns.append(tuple[0])
  return nltk_nouns

In [9]:
def dynamicPlantumlCode(text):
  
  doc = sent_tokenize(text)
 
  f = open('d_model_specs.txt','w')

  # Needed to state what type of diagram is being constructed (State Diagram)
  f.write("state Diagram {\nhide empty description \n[*]--> Start\n")
  # Iterate sentence-by-sentence
  for sent in doc:
    # Call Tokenize and POS Funtion 
    token_sentence, pos_tag_token = tokenAndPOS_Tags(sent)

    # Call Noun Extraction Function 
    nltk_nouns = nounExtraction(pos_tag_token)    

    # if the sentence include condition and true, construct the 
    # sentence: input- current state - transition state
    # code: current state-->transition state:[input]
    if (("condition" in token_sentence) and ("true" in token_sentence) is True): 
      f.write(f"{nltk_nouns[1]}-->{nltk_nouns[2]}:[{nltk_nouns[0]}]\n")
    
    # if the sentence include condition and false, construct the 
    # sentence: input- current state - transition state
    # code: current state --> transition state: not[input]
    elif ("condition" in token_sentence) and ("false" in token_sentence):
      f.write(f"{nltk_nouns[1]}-->{nltk_nouns[2]}:not[{nltk_nouns[0]}]\n")

    # if the sentence include variable, construct the
    # sentence: variable name- variable assignment- variable location
    # code: variable location:variable name= variable assignment 
    elif "variable" in token_sentence:
      f.write(f"{nltk_nouns[2]}:{nltk_nouns[0]}={nltk_nouns[1]}\n")

    # if the sentence include event, construct the
    # sentence: event name- current state- transition state
    # code: current state-->transition state:event[event name]
    elif "event" in token_sentence: 
      f.write(f"{nltk_nouns[1]}-->{nltk_nouns[2]}:event[{nltk_nouns[0]}]\n")

  # needed for creating state diagrams
  f.write("}")
  f.close()


### Generting the Dynamic Behavior Diagram

In [10]:
dynamicPlantumlCode(modified_text)
server = PlantUML(url='http://www.plantuml.com/plantuml/img/',
                          basic_auth={},
                          form_auth={}, http_opts={}, request_opts={})

server.processes_file(abspath('d_model_specs.txt')) 

True